# 미세 조정 분류 예

야구와 하키라는 두 스포츠를 구분하기 위해 ada 분류기를 미세 조정할 것입니다.

In [23]:
from sklearn.datasets import fetch_20newsgroups
import pandas as pd
import openai

categories = ['rec.sport.baseball', 'rec.sport.hockey']
sports_dataset = fetch_20newsgroups(subset='train', shuffle=True, random_state=42, categories=categories)

 ## 데이터 탐색
뉴스 그룹 데이터 세트는 sklearn을 사용하여 로드할 수 있습니다. 먼저 데이터 자체를 살펴보겠습니다.

In [25]:
print(sports_dataset['data'][0])

From: dougb@comm.mot.com (Doug Bank)
Subject: Re: Info needed for Cleveland tickets
Reply-To: dougb@ecs.comm.mot.com
Organization: Motorola Land Mobile Products Sector
Distribution: usa
Nntp-Posting-Host: 145.1.146.35
Lines: 17

In article <1993Apr1.234031.4950@leland.Stanford.EDU>, bohnert@leland.Stanford.EDU (matthew bohnert) writes:

|> I'm going to be in Cleveland Thursday, April 15 to Sunday, April 18.
|> Does anybody know if the Tribe will be in town on those dates, and
|> if so, who're they playing and if tickets are available?

The tribe will be in town from April 16 to the 19th.
There are ALWAYS tickets available! (Though they are playing Toronto,
and many Toronto fans make the trip to Cleveland as it is easier to
get tickets in Cleveland than in Toronto.  Either way, I seriously
doubt they will sell out until the end of the season.)

-- 
Doug Bank                       Private Systems Division
dougb@ecs.comm.mot.com          Motorola Communications Sector
dougb@nwu.edu       

In [26]:
sports_dataset.target_names[sports_dataset['target'][0]]


'rec.sport.baseball'

In [28]:
len_all, len_baseball, len_hockey = len(sports_dataset.data), len([e for e in sports_dataset.target if e == 0]), len([e for e in sports_dataset.target if e == 1])
print(f"Total examples: {len_all}, Baseball examples: {len_baseball}, Hockey examples: {len_hockey}")

Total examples: 1197, Baseball examples: 597, Hockey examples: 600


야구 카테고리의 샘플은 위에서 볼 수 있습니다. 메일링 리스트에 대한 이메일입니다. 총 1197개의 예가 있으며 두 스포츠 간에 균등하게 분할되어 있음을 알 수 있습니다.

## 데이터 준비
프롬프트 및 완료를 위한 열이 있는 pandas 데이터 프레임으로 데이터 세트를 변환합니다. 프롬프트에는 메일링 리스트의 이메일이 포함되어 있으며 완료는 스포츠 이름(하키 또는 야구)입니다. 데모 목적과 미세 조정 속도를 위해 300개의 예만 사용합니다. 실제 사용 사례에서는 예제가 많을수록 성능이 향상됩니다.

In [29]:
import pandas as pd

labels = [sports_dataset.target_names[x].split('.')[-1] for x in sports_dataset['target']]
texts = [text.strip() for text in sports_dataset['data']]
df = pd.DataFrame(zip(texts, labels), columns = ['prompt','completion']) #[:300]
df.head()

,prompt,completion
0,From: dougb@comm.mot.com (Doug Bank)\nSubject:...,baseball
1,From: gld@cunixb.cc.columbia.edu (Gary L Dare)...,hockey
2,From: rudy@netcom.com (Rudy Wade)\nSubject: Re...,baseball
3,From: monack@helium.gas.uug.arizona.edu (david...,hockey
4,Subject: Let it be Known\nFrom: <ISSBTL@BYUVM....,baseball


야구와 하키는 모두 단일 토큰입니다. 데이터 세트를 jsonl 파일로 저장합니다.

In [30]:
df.to_json("sport2.jsonl", orient='records', lines=True)

### 데이터 준비 도구
이제 미세 조정 전에 데이터 세트에 대한 몇 가지 개선 사항을 제안하는 데이터 준비 도구를 사용할 수 있습니다. 도구를 시작하기 전에 최신 데이터 준비 도구를 사용하고 있는지 확인하기 위해 openai 라이브러리를 업데이트합니다. 모든 제안을 자동으로 수락하는 `-q`를 추가로 지정합니다.

In [ ]:
!pip install --upgrade openai

In [31]:
!openai tools fine_tunes.prepare_data -f sport2.jsonl -q

Analyzing...

- Your file contains 1197 prompt-completion pairs
- Based on your data it seems like you're trying to fine-tune a model for classification
- For classification, we recommend you try one of the faster and cheaper models, such as `ada`
- For classification, you can estimate the expected model performance by keeping a held out dataset, which is not used for training
- There are 11 examples that are very long. These are rows: [134, 200, 281, 320, 404, 595, 704, 838, 1113, 1139, 1174]
For conditional generation, and for classification the examples shouldn't be longer than 2048 tokens.
- Your data does not contain a common separator at the end of your prompts. Having a separator string appended to the end of the prompt makes it clearer to the fine-tuned model where the completion should begin. See https://beta.openai.com/docs/guides/fine-tuning/preparing-your-dataset for more detail and examples. If you intend to do open-ended generation, then you should leave the prompts empty

이 도구는 데이터 세트에 대한 몇 가지 개선 사항을 유용하게 제안하고 데이터 세트를 훈련 및 검증 세트로 분할합니다.

입력 텍스트가 중지되었으며 이제 클래스를 예측해야 함을 모델에 알리려면 프롬프트와 완료 사이의 접미사가 필요합니다. 각 예에서 동일한 구분 기호를 사용하기 때문에 모델은 구분 기호 다음에 나오는 야구 또는 하키를 예측하기 위한 것임을 학습할 수 있습니다.
대부분의 단어 토큰은 공백 접두사로 토큰화되므로 완성 시 공백 접두사가 유용합니다.
또한 이 도구는 이것이 분류 작업일 가능성이 있음을 인식하여 데이터 세트를 학습 및 검증 데이터 세트로 분할할 것을 제안했습니다. 이를 통해 새로운 데이터에 대한 예상 성능을 쉽게 측정할 수 있습니다.

## 미세 조정
이 도구는 다음 명령을 실행하여 데이터 세트를 훈련할 것을 제안합니다. 이것은 분류 작업이므로 제공된 검증 세트의 일반화 성능이 분류 사용 사례에 대해 무엇인지 알고 싶습니다. 이 도구는 분류 메트릭을 계산하기 위해 `--compute_classification_metrics --classification_positive_class " 야구"`를 추가할 것을 제안합니다.

CLI 도구에서 제안된 명령을 간단히 복사할 수 있습니다. 우리는 특히 `-m ada`를 추가하여 더 저렴하고 빠른 ada 모델을 미세 조정합니다. 이는 일반적으로 분류 사용 사례에서 더 느리고 더 비싼 모델과 성능이 비슷합니다.

In [ ]:
!openai api fine_tunes.create -t "sport2_prepared_train.jsonl" -v "sport2_prepared_valid.jsonl" --compute_classification_metrics --classification_positive_class " baseball" -m ada

모델은 약 10분 안에 성공적으로 학습됩니다. 모델 이름이 'ada:ft-openai-2021-07-30-12-26-20'인 것을 볼 수 있으며 이는 추론에 사용할 수 있습니다.

### [고급] 결과 및 예상 모델 성능
이제 결과 파일을 다운로드하여 보류된 유효성 검사 세트에서 예상되는 성능을 관찰할 수 있습니다.

In [ ]:
!openai api fine_tunes.results -i ft-gpH6MKybidrK6U1Kd4blHwiP > result.csv

In [ ]:
results = pd.read_csv('result.csv')
results[results['classification/accuracy'].notnull()].tail(1)

정확도는 99.6%에 이릅니다. 아래 플롯에서 훈련 실행 중에 검증 세트의 정확도가 어떻게 증가하는지 확인할 수 있습니다.

In [ ]:
results[results['classification/accuracy'].notnull()]['classification/accuracy'].plot()

## 모델 사용
이제 모델을 호출하여 예측을 얻을 수 있습니다.

In [ ]:
test = pd.read_json('sport2_prepared_valid.jsonl', lines=True)
test.head()

미세 조정 중에 사용한 프롬프트 다음에 동일한 구분 기호를 사용해야 합니다. 이 경우 `\n\n###\n\n`입니다. 우리는 분류에 관심이 있기 때문에 가능한 한 낮은 온도를 원하고 모델의 예측을 결정하기 위해 하나의 토큰 완료만 필요합니다.

In [ ]:
ft_model = 'ada:ft-personal-2022-05-31-12-30-20'
res = openai.Completion.create(model=ft_model, prompt=test['prompt'][0] + '\n\n###\n\n', max_tokens=1, temperature=0)
res['choices'][0]['text']


로그 확률을 얻기 위해 완료 요청에 logprobs 매개변수를 지정할 수 있습니다.

In [ ]:
res = openai.Completion.create(model=ft_model, prompt=test['prompt'][0] + '\n\n###\n\n', max_tokens=1, temperature=0, logprobs=2)
res['choices'][0]['logprobs']['top_logprobs'][0]

모델이 하키를 야구보다 훨씬 더 가능성이 높은 것으로 예측한다는 것을 알 수 있습니다. 이는 올바른 예측입니다. log_probs를 요청하여 각 클래스에 대한 예측(로그) 확률을 볼 수 있습니다.

### 일반화
흥미롭게도 미세 조정된 분류기는 매우 다양합니다. 다른 메일링 리스트로 보내는 이메일에 대해 교육을 받았음에도 불구하고 트윗을 성공적으로 예측합니다.

In [ ]:
sample_hockey_tweet = """Thank you to the 
@Canes
 and all you amazing Caniacs that have been so supportive! You guys are some of the best fans in the NHL without a doubt! Really excited to start this new chapter in my career with the 
@DetroitRedWings
 !!"""
res = openai.Completion.create(model=ft_model, prompt=sample_hockey_tweet + '\n\n###\n\n', max_tokens=1, temperature=0, logprobs=2)
res['choices'][0]['text']

In [ ]:
sample_baseball_tweet="""BREAKING: The Tampa Bay Rays are finalizing a deal to acquire slugger Nelson Cruz from the Minnesota Twins, sources tell ESPN."""
res = openai.Completion.create(model=ft_model, prompt=sample_baseball_tweet + '\n\n###\n\n', max_tokens=1, temperature=0, logprobs=2)
res['choices'][0]['text']